In [4]:
# Kütüphaneler

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.metrics import mean_squared_error , r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale     # standardizasoyon işlemleri
from sklearn.preprocessing import StandardScaler    # standardizasoyon işlemleri
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn import neighbors
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

import xgboost
from xgboost import XGBRegressor


from warnings import filterwarnings
filterwarnings("ignore")     # önceki bölümlerde çalıştırdığımızda çıkan gereksiz uyarı mesajları çıkmasın diye

In [3]:
!pip install xgboost

   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB 660.6 kB/s eta 0:03:10
   ---------------------------------------- 0.7/124.9 MB 7.6 MB/s eta 0:00:17
    --------------------------------------- 2.3/124.9 MB 16.0 MB/s eta 0:00:08
   - -------------------------------------- 3.8/124.9 MB 20.4 MB/s eta 0:00:06
   - -------------------------------------- 4.0/124.9 MB 21.2 MB/s eta 0:00:06
   - -------------------------------------- 4.6/124.9 MB 16.3 MB/s eta 0:00:08
   - -------------------------------------- 6.2/124.9 MB 18.9 MB/s eta 0:00:07
   -- ------------------------------------- 7.5/124.9 MB 20.8 MB/s eta 0:00:06
   -- ------------------------------------- 7.5/124.9 MB 20.8 MB/s eta 0:00:06
   -- ------------------------------------- 8.6/124.9 MB 19.1 MB/s eta 0:00:07
   --- ------------------------------------ 10.1/124.9 MB 20.3 MB/s eta 0:00:06
   --- ------------------------------------ 10.7/124.9 MB 2


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
df = pd.read_csv("C:\Makine Öğrenmesi-2\Hitters.csv")
df = df.dropna()

dms = pd.get_dummies(df[["League" , "Division" , "NewLeague"]])

y = df["Salary"]

x__ = df.drop(["Salary" , "League" , "Division" , "NewLeague"] , axis=1).astype("float64")
x = pd.concat([x__ , dms[["League_N" , "Division_W" , "NewLeague_N"]]] , axis=1)

x_train , x_test , y_train , y_test = train_test_split(x , y , test_size=0.25 , random_state=42)

# MODEL

In [8]:
xgb_model = XGBRegressor().fit(x_train , y_train)

In [7]:
xgb_model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

# TAHMİN

In [9]:
y_pred = xgb_model.predict(x_test)
np.sqrt(mean_squared_error(y_test , y_pred))

366.3863437634965

# MODEL TUNING (DOĞRULAMA)

***Önemli Parametreler:***
- learning_rate: overfitting'i engellemek adına kullanılan bir hiperparametredir. Daraltma adım boyutunu ifade eder.
- colsample_bytree: oluşturulacak olan ağaçlarda değişkenlerden alınacak olan alt küme oranı 

In [10]:
xgb_params = {"learning_rate": [0.1, 0.01, 0.5] , "max_depth": [2,3,4] , "n_estimators": [100,200,500] , "colsample_bytree": [0.4 , 0.7 , 1]}

In [12]:
xgb_cv_model = GridSearchCV(xgb_model , xgb_params , cv=10 , n_jobs=-1 , verbose=2).fit(x_train , y_train)

Fitting 10 folds for each of 81 candidates, totalling 810 fits


In [13]:
xgb_cv_model.best_params_

{'colsample_bytree': 0.4,
 'learning_rate': 0.5,
 'max_depth': 3,
 'n_estimators': 100}

In [28]:
xgb_tuned = XGBRegressor(colsample_bytree= 0.4 , learning_rate= 0.5 , max_depth= 3 , n_estimators= 100).fit(x_train , y_train)

In [29]:
y_pred = xgb_tuned.predict(x_test)
np.sqrt(mean_squared_error(y_pred , y_test))

374.29091564195204